In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/FUENTES/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/DATOS/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics, model_selection

import time
#from matplotlib import pyplot as plt
from matplotlib import pylab as plt
from IPython import display


In [3]:
df = pd.read_csv(DATOS_DIR + 'iris.csv')
df.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Ejemplos utilizados en las diapositivas

Usaremos una red neuronal maultipercpetrón para clasificar flores de iris.
Se dispone de 4 atributos numéricos de entrada y una etiqueta de clase con 3 valores posibles. 
A fin de ejemplificar los cálculo, se han seleccionado algunos ejemplos del data set original luego de aplicar una normalización utilizando media y desvío.

In [4]:
X = np.array([[-1.73,-0.05,-1.38,-1.31], 
              [-0.37,-1.62, 0.22, 0.18],
              [ 1.11,-0.05, 0.93, 1.54],
              [-0.99, 0.39,-1.44,-1.31],
              [ 1.73, 1.29, 1.46, 1.81]])
Y=np.array([[1,0,0], 
            [0,1,0],
            [0,0,1],
            [1,0,0],
            [0,0,1]])

X es una matriz de NxM donde N es la cantidad de ejemplos y M es la cantidad de atributos
Y es una matrz de NxC donde N es la cantidad de ejemplos y C la cantidad de valores del atributo de clase.

#### Matrices de pesos

In [5]:
W1 = np.array([[ 0.15, -0.13,  0.23, -0.45],
               [-0.29, -0.41, -0.19,  0.37]])
b1 = np.array([[-0.45],
               [-0.10]])
W2 = np.array([[ -4.79, -0.99],
               [ 0.38,  0.09],
               [ 2.40,  -0.38]])
b2 = np.array([[-0.29],
               [-0.17],
               [-0.27]])

**W1** es la matriz de pesos que une la capa de entrada con la capa oculta. Tiene dimensión OxM siendo O la cantidad de neuronas ocultas y M la cantidad de atributos.
**b1** es un vector columna con O elementos, es decir con tantos valores como neuronas ocultas haya y representa el sesgo de cada neurona oculta.

**W2** contiene los pesos de los arcos que van de las neuronas ocultas a las neuronas de la capa de salida. Su dimensión es SxO siendo S la cantidad de neuronas de la cpaa de salida y O la cantidad de neuronas ocultas.
**b2** es un vector columna que contiene los sesgos de cada neurona de la capa de salida, es decir que tiene S elementos.

#### Notación para acceder a cada ejemplo como una matriz de una sola fila

Utilizando esta notación de matriz es posible trasponer los vectores

In [6]:
e = 0  # nro. de ejemplo a introducir a la red
xi = X[e:e+1, :]  #fila e de X
yi = Y[e:e+1, :]  # respuesta esperada para el ejemplo e
print("xi = ",xi)
print("xi.shape = ", xi.shape)
print("\nyi = ", yi)
print("yi.shape = ", yi.shape)


xi =  [[-1.73 -0.05 -1.38 -1.31]]
xi.shape =  (1, 4)

yi =  [[1 0 0]]
yi.shape =  (1, 3)


### Propagamos el ejemplo e hacia adelante

#### Salidas de la capa oculta con función de activación TANH

In [7]:
netasH = W1 @ xi.T + b1
print("netaH \n", netasH)
salidasH = 2.0/(1+np.exp(-2*netasH))-1
print("salidasH = \n", salidasH)

netaH 
 [[-0.4309]
 [ 0.1997]]
salidasH = 
 [[-0.40607318]
 [ 0.19708699]]


#### Salida de la red (capa de salida) con función de activación SIGMOID

In [8]:
netasO =  W2 @ salidasH + b2
print("netaO \n", netasO)
salidasO = 1.0/(1+np.exp(-netasO))
print("salidaO = \n", salidasO)

netaO 
 [[ 1.4599744 ]
 [-0.30656998]
 [-1.31946868]]
salidaO = 
 [[0.81152876]
 [0.42395219]
 [0.2109067 ]]


#### Error de cada neurona de la capa de salida 

In [9]:
print("salidaO = \n", salidasO)
ErrorSalida = yi.T-salidasO
print("\nErrorSalida = \n", ErrorSalida)


salidaO = 
 [[0.81152876]
 [0.42395219]
 [0.2109067 ]]

ErrorSalida = 
 [[ 0.18847124]
 [-0.42395219]
 [-0.2109067 ]]


#### Calculamos los factores de corrección de los pesos

In [10]:
# la función de activación de la capa de salida es SIGMOID
deriv_FunO = salidasO * (1-salidasO)
print("deriv_FunO = \n", deriv_FunO)
deltaO = ErrorSalida * deriv_FunO
print("deltaO = \n", deltaO)

# la función de atcivación de la capa oculta es TANH
deriv_FunH = (1 - salidasH**2)
print("\nderiv_FunH = \n", deriv_FunH)

errorH = W2.T @ deltaO
print("\nW2.T @ deltaO = \n", errorH)

deltaH = deriv_FunH * (W2.T @ deltaO)
print("deltasH = \n", deltaH)

print("\nW2.T = \n", np.round(W2.T,2))

deriv_FunO = 
 [[0.15294983]
 [0.24421673]
 [0.16642507]]
deltaO = 
 [[ 0.02882664]
 [-0.10353622]
 [-0.03510016]]

deriv_FunH = 
 [[0.83510457]
 [0.96115672]]

W2.T @ deltaO = 
 [[-0.26166378]
 [-0.02451858]]
deltasH = 
 [[-0.21851662]
 [-0.02356619]]

W2.T = 
 [[-4.79  0.38  2.4 ]
 [-0.99  0.09 -0.38]]


#### Corrigiendo los pesos

In [11]:
# corregir los pesos
alfa=0.1

W1 = W1 + alfa * deltaH @ xi 
b1 = b1 + alfa * deltaH 
W2 = W2 + alfa * deltaO @ salidasH.T 
b2 = b2 + alfa * deltaO 

print("W1 = \n", np.round(W1,2))
print("b1 = \n", np.round(b1,2))
print("W2 = \n", np.round(W2,2))
print("b2 = \n", np.round(b2,2))

W1 = 
 [[ 0.19 -0.13  0.26 -0.42]
 [-0.29 -0.41 -0.19  0.37]]
b1 = 
 [[-0.47]
 [-0.1 ]]
W2 = 
 [[-4.79 -0.99]
 [ 0.38  0.09]
 [ 2.4  -0.38]]
b2 = 
 [[-0.29]
 [-0.18]
 [-0.27]]


#### Calculamos la salida de la red para el mismo ejemplo luego de corregir los pesos

In [12]:
netasH = W1 @ xi.T + b1
salidasH = 2.0/(1+np.exp(-2*netasH))-1
netasO =  W2 @ salidasH + b2
salidasO = 1.0/(1+np.exp(-netasO))
print("salidaO = \n", salidasO)
ErrorSalidaNew = yi.T-salidasO
print("ErrorSalida = \n", ErrorSalidaNew)

salidaO = 
 [[0.89080126]
 [0.40850633]
 [0.16423515]]
ErrorSalida = 
 [[ 0.10919874]
 [-0.40850633]
 [-0.16423515]]


#### Comparando el error antes y después de corregir los pesos

In [13]:
print("Error inicial = ", np.sum(ErrorSalida**2))
print("Error luego de la correccion = ", np.sum(ErrorSalidaNew**2))

Error inicial =  0.25973850457967945
Error luego de la correccion =  0.2057749693024508
